In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from skimage import io, color
from skimage.io import imread_collection
from ast import Yield
import matplotlib.pyplot as plt
import glob
import cv2
import os

In [ ]:
height= 80
width=200

In [ ]:
train_path = 'data/train/*.png'
val_path = 'data/validation/*.png'
y_path = 'data/train.csv'
y_val_path ='data/validation.csv'
Y_train = pd.read_csv(y_path, dtype=str)
Y_val = pd.read_csv(y_val_path, dtype=str)
train = imread_collection(train_path)
val = imread_collection(val_path)
Y_train_digits= []
Y_val_digits = []
Y_train['Label'] = Y_train['Label'].astype(str)
Y_val['Label'] = Y_val['Label'].astype(str)


c =0
window =33
X_train=[]
X_val=[]
for image in train:
    
    for x in range (1,6):
        roi = image[0:80, x+window*(x-1):x+window*x]
        grayscale_image = color.rgb2gray(roi)
        flattened_image = grayscale_image.ravel()
        normalized_image = (flattened_image - flattened_image.min()) / (flattened_image.max() - flattened_image.min())
        Y_train_digits.append(Y_train['Label'].iloc[c][x-1])
        X_train.append(roi)
        
    c+=1
c =0

for image in val:

    for x in range (1,6):
        roi = image[0:80, x+window*(x-1):x+window*x]
        grayscale_image = color.rgb2gray(roi)
        flattened_image = grayscale_image.ravel()
        normalized_image = (flattened_image - flattened_image.min()) / (flattened_image.max() - flattened_image.min())
        Y_val_digits.append(Y_val['Label'].iloc[c][x-1])
        X_val.append(roi)
        
    c+=1
    
X_train = np.array(X_train).reshape(-1, height, width, 1)
X_val =np.array(X_val).reshape(-1, height, width, 1)
Y_train_digits = pd.DataFrame(Y_train_digits, columns=['Label'])
Y_val_digits = pd.DataFrame(Y_val_digits, columns=['Label'])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    label = Y_train.iloc[i]['Label']
    plt.title(f'Label is {label}')


plt.show()

In [ ]:
plt.figure(figsize=(15,20))
for i in range(50):
    plt.subplot(10,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_val[i])
    label = Y_val_digits.iloc[i]['Label']
    plt.title(f'Label is {label}')


plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Y_train_labels = Y_train_digits['Label'].to_numpy()
y_combine = LabelEncoder().fit_transform(Y_train_labels)
y_one_hot = OneHotEncoder().fit_transform(y_combine.reshape(len(y_combine),1))

In [93]:
from keras.utils import to_categorical
Y_train_one_hot = to_categorical(Y_train_digits)
Y_val_one_hot = to_categorical(Y_val_digits)

In [98]:
Y_train_one_hot.shape

(50000, 10)

In [88]:
X_train= np.array(X_train)
print(X_train.shape)
print(X_val.shape)


(24750, 80, 200, 1)
(2475, 80, 200, 1)


In [90]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(80,200,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [91]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [96]:
model.fit(X_train, Y_train_one_hot, validation_data=(X_val, Y_val_one_hot), epochs=3)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 24750
'y' sizes: 50000
